Si provi ad allenare il regressore sul dataset Boston senza effettuare la normalizzazione dei dati. Se necessario si modifichino il learning rate e il numero di epoche per portare il modello a convergenza. Si notino differenze durante il training? Il modello allenato è migliore o peggiore? Si monitorino le curve di training e test mediante tensorboard.

In [ ]:
from torch import nn 
import numpy as np
from sklearn.datasets import fetch_openml
boston = fetch_openml(name='boston', version=1) # Serve per ottenere dei datasets da openml

X = boston.data.to_numpy().astype(float) # Convertiamo un dataframe in un array in modo tale da poter fare la shape
Y = boston.target.to_numpy().astype(float) # stessa cosa


In [ ]:
import torch
np.random.seed(123)
torch.random.manual_seed(123)
idx = np.random.permutation(len(X))

X = X[idx]
Y = Y[idx]

In [ ]:
X_training = torch.Tensor(X[50:])
Y_training = torch.Tensor(Y[50:])

X_testing = torch.Tensor(X[:50])
Y_testing = torch.Tensor(Y[:50])


In [ ]:
theta = torch.Tensor(13) # Creiamo un tensore di 13 unità
theta_0= torch.Tensor(1) # Tensore di una unità (bias)

#impostiamo required_grad
theta.requires_grad_(True)
theta_0.requires_grad_(True)

In [ ]:
def linear_regression(input,theta,theta_0):
    return input.mul(theta).sum(1)+theta_0
    #.mul(theta) fa un prodotto elemento per elemento tra ogni riga dell'input e theta
    #.sum(1) Dopo il prodotto elemento per elemento, sommi lungo l’asse 1 (cioè per riga) 
    # + theta_0 Aggiungi il bias a ciascuna predizione.

In [ ]:
def loss(input, target):
        return ((input-target)**2).mean()

In [ ]:
from  torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter('logs/Esercizio1')

class LinearRegressor(nn.Module):
    def __init__(self, in_size, out_size):
        """Costruisce un regressore logistico.
        Input:
            in_size: numero di feature in input (es. 13)
            out_size: numero di elementi in output (es. 1)"""
        super(LinearRegressor, self).__init__() #Richiamo del costruttore della superclasse
        # Questo passo è necessario per abilitare alcuni meccanismi automatici dei moduli di Pytorch
        self.linear = nn.Linear(in_size,out_size)

    def forward(self,x):
        """Definisce come processare l'input x"""
        result = self.linear(x)

        return result

In [ ]:
lr = 0.000001
epochs = 1000000

# Modello
reg = LinearRegressor(13, 1)
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(reg.parameters(), lr=lr)

train_losses = []
test_losses = []

# Training loop
for e in range(epochs):
    reg.train()
    output = reg(X_training)
    l = criterion(output.view(-1), Y_training)
    train_losses.append(l.item())

    writer.add_scalar('loss/train', train_losses[-1], global_step=e)
    l.backward()
    optimizer.step()
    optimizer.zero_grad()

    reg.eval()
    with torch.set_grad_enabled(False):
        y_test = reg(X_testing)
        l = criterion(y_test.view(-1), Y_testing)
        test_losses.append(l.item())
                            
        writer.add_scalar('loss/test', test_losses[-1], global_step=e)

    if (e+1)%100 == 0:
        print(f'[Epoc {e+1}]\t train loss: {np.round(train_losses[-1], 2)}    test loss: {np.round(test_losses[-1], 2)}')